In [13]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

%run Metric.ipynb
%run Section.ipynb
%run SectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb

cate_single_level_sections = 'single_level'
cate_resonate_on_week_and_day = 'resonate_WD'
    
class SectionAnalyser:
    
    def __init__(self, scale, short, mid):
        self.symbols = get_symbols(scale)
        
        self.scale = scale
        self.short = short
        self.mid = mid
    
    def build_sections_file_path(self, cate, level):
        return section_dir + cate +'_'+str(self.short) +'_'+str(self.mid) +'_'+level +'_'+str(self.scale)+ '_all_sections_' + section_version+'.txt'
        
    def get_all_sections_of_all_symbols(self, level):
        file_path = self.build_sections_file_path(cate_single_level_sections,level)
        
        all_sections_df = pd.DataFrame(columns=section_columns)

        if(os.path.exists(file_path)):
            all_sections_df = pd.read_csv(file_path)
        else:
            for i in range(0, len(self.symbols)):
                sectionManager = SectionManager(self.symbols[i], level, self.short, self.mid)
                temp_section = sectionManager.get_all_cross_sections()

                all_sections_df = pd.concat([temp_section,all_sections_df], ignore_index=True, sort=False)

            all_sections_df.to_csv(file_path, index=0)
        
        return all_sections_df

    def get_price_incre_percent_report(self, sections_df):
        for threshold in [10,20,30,40,50,60,70,80,90]:
            over_count = sections_df[sections_df['price_incre_per'] > threshold].shape[0]
            total = sections_df.shape[0]

            if(total>0):
                rate = 100*over_count/total
                print('threshold '+ str(threshold) + ', over count size:'+ str(over_count) +
                      ' total size:'+str(total) + ' rate ' +str(rate))
        
    
    def get_ampl_report(self, sections_df):
        for threshold in [60,70,80,90,100]:
            ampl_count = sections_df[sections_df['ampl'] > threshold].shape[0]
            total = sections_df.shape[0]

            if(total>0):
                rate = 100*ampl_count/total
                print('threshold '+ str(threshold) + ', ampl size:'+ str(ampl_count) +
                      ' total size:'+str(total) + ' rate ' +str(rate))

                
    def build_cross_resonate_on_week_and_day(self):
        resonate_df = pd.DataFrame(columns=resonate_cross_columns)
        
        week_sections_df = self.get_all_sections_of_all_symbols(level_week)
        
        symbols_set = week_sections_df['symbol'].drop_duplicates()

        progress = 0
        total_count = symbols_set.shape[0]
        
        print('find all cross sections...')
        for symbol in symbols_set:
            sectionManager = SectionManager(symbol, level_day, self.short, self.mid)
            dates = week_sections_df[(week_sections_df['symbol']==symbol)]['s_date']

            progress += 1
            if(progress % 4 ==0):
                print(str(round(100*(progress/total_count),2))+'%', end=' ')
            
            for date in dates:
                w_section = week_sections_df[(week_sections_df['symbol']==symbol) & (week_sections_df['s_date']==date)]

                w_start_date = w_section['s_date'].values[0]
                w_end_date = w_section['e_date'].values[0]
                
                w_ma_short_direction = w_section['ma_short_direction'].values[0]
                w_ma_mid_direction = w_section['ma_mid_direction'].values[0]

                d_sections = sectionManager.find_sections_between(w_start_date, w_end_date)

                for index,row in d_sections.iterrows():
                    d_s_date = row['s_date']
                    d_e_date = row['e_date']
                    d_ampl = row['ampl']
                    d_boll = row['boll']
                    
                    d_price_incre_per = row['price_incre_per']
                    d_ma_short_direction = row['ma_short_direction']
                    d_ma_mid_direction = row['ma_mid_direction']
                    d_break_mid_ma_price_incre_per = row['break_mid_ma_price_incre']

                    resonate_df.loc[resonate_df.shape[0]+1] = [symbol,w_start_date, w_end_date,
                                                               d_s_date,d_e_date,d_ampl,d_boll,d_price_incre_per,
                                                               d_ma_short_direction,d_ma_mid_direction,
                                                               w_ma_short_direction,w_ma_mid_direction,
                                                               d_break_mid_ma_price_incre_per
                                                              ]

        return resonate_df
        
                
    def get_cross_resonate_on_week_and_day(self):
        file_path = self.build_sections_file_path(cate_resonate_on_week_and_day,'WD')
        
        resonate_df = pd.DataFrame(columns=resonate_cross_columns)
        
        if(os.path.exists(file_path)):
            resonate_df = pd.read_csv(file_path)
        else:
            resonate_df = self.build_cross_resonate_on_week_and_day()
        
        return resonate_df
